In [1]:
%reload_ext autoreload
%autoreload 2

## Setup

In [2]:
from math import isclose
from ipyexperiments import IPyExperimentsCPU
import ipyexperiments
import re
from utils.text import *

In [3]:
"""test_version"""
assert ipyexperiments.__version__, "version check"

'test_version'

## Basic container test


In [4]:
%%capture output
exp1 = IPyExperimentsCPU(cl_enable=False)

x1 = consume_cpu_ram_128mb()
x2 = consume_cpu_ram_128mb()
check_defined(['x1', 'x2'], locals())

#exp1.finish()
del exp1
check_undefined(['x1', 'x2'], locals())

In [ ]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8149  31588 MB   0.51% 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     255     255 MB ( 99.99%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8147  31588 MB   0.51% 
| 
| 
| 



## Test stats data and selective vars preservation

In [ ]:
%%capture output
exp2 = IPyExperimentsCPU(cl_enable=False) 
x1 = consume_cpu_ram_128mb()

cpu_data = exp2.data
assert cpu_data
print(cpu_data)

exp2.keep_var_names('cpu_data')

x2 = consume_cpu_ram_128mb() 

cpu_data = exp2.data
print(cpu_data)

check_defined(['x1', 'x2', 'cpu_data'], locals())
    
cpu_data_final = exp2.finish() # finish experiment
print("\nNumerical data:\n", cpu_data_final)

check_defined(['cpu_data_final'], locals())
check_undefined(['x1', 'x2'], locals())

In [ ]:
output = str(output)
print_output(output)
check_reclaimed(output)
check_data(output, cpu_data_final)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8148  31588 MB   0.51% 
| 
| 
| IPyExperimentMemory(consumed=134078464, reclaimed=0, available=8409972736)
| IPyExperimentMemory(consumed=268435456, reclaimed=0, available=8276320256)
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| Kept:    cpu_data
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     256     255 MB ( 99.98%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8148  31588 MB   0.51% 
| 
| 
| 
| Numerical data:
|  IPyExperimentMemory(consumed=268435456, reclaimed=268369920, available=8543850496)
| 



## Using the context manager

If you want to put all cells into one, you could simplify the experiment even further by using its context manager.

In [ ]:
%%capture output
with IPyExperimentsCPU(cl_enable=False):
    x1 = consume_cpu_ram_128mb()
    x2 = consume_cpu_ram_128mb()
    
check_undefined(['x1', 'x2'], locals())

In [ ]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8148  31588 MB   0.51% 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     256     256 MB (100.00%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8148  31588 MB   0.51% 
| 
| 
| 



In [ ]:
%%capture output
with IPyExperimentsCPU(cl_enable=False) as exp: 
    x1 = consume_cpu_ram_128mb()
    z = "some data"
    x2 = consume_cpu_ram_128mb()
    exp.keep_var_names('z')
print(z)
check_defined(['z'], locals())
check_undefined(['x1', 'x2'], locals())

In [ ]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8148  31588 MB   0.51% 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| Kept:    z
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     256     256 MB (100.00%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8147  31588 MB   0.51% 
| 
| 
| some data
| 



## Deleting previously defined variables

Here we test that we **cannot** correctly detect and delete variables used in the scope of the experiment, but which were already defined prior to the experiment.

In [ ]:
%%capture output
x3 = 10
x4 = 20
with IPyExperimentsCPU(cl_enable=False): 
    x1 = consume_cpu_ram_128mb()
    x2 = consume_cpu_ram_128mb()
    x3 = 10
    x4 = 10000
# x3 is the same value, so we can't tell the difference whether it was created before the experiment or not
# x4 is different from previous value, but it could have been modified by some function indirectly and not used directly in the experiment
check_defined(['x3', 'x4'], locals())
# x3 is newly defined, so it gets deleted
check_undefined(['x1', 'x2'], locals())

In [ ]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8148  31588 MB   0.51% 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     256     255 MB ( 99.99%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8146  31588 MB   0.51% 
| 
| 
| 



Same but with the CellLogger enabled

In [ ]:
%%capture output
x3 = 10
x4 = 20
with IPyExperimentsCPU(cl_enable=True): 
    x1 = consume_cpu_ram_128mb()
    x2 = consume_cpu_ram_128mb()
    x3 = 10
    x4 = 10000
# x3 is the same value, so we can't tell the difference whether it was created before the experiment or not
# x4 is different from previous value, but it could have been modified by some function indirectly and not used directly in the experiment
check_defined(['x3', 'x4'], locals())
# x3 is newly defined, so it gets deleted
check_undefined(['x1', 'x2'], locals())

In [ ]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8146  31588 MB   0.51% 
| 
| 
| ･ RAM: △Consumed △Peaked  Used Total | Exec time 0.057s
| ･ CPU:       256       0      417 MB   |
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM:  Consumed     Reclaimed
| CPU:     256     255 MB (100.00%)
| 
| *** Current state:
| RAM:  Used  Free  Total      Util
| CPU:   161  8145  31588 MB   0.51% 
| 
| 
| 



In [ ]:
%%javascript # prevent committing an unsaved notebook
IPython.notebook.save_notebook()